To use this notebook, you should have make_input_file.py and galfitwrap.py in the same directory as this notebook

It's also recommended to have the cgmsquared_full_galaxy_table.fits file and both the flattened and unflattened image files in the same directory as this notebook. This means you can simply enter the name of the file wherever the notebook calls for a file path.

In [ ]:
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.table import Table

from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

import WerkSQuAD_galfit_functions as galf
import Display_galaxies_V4 as dis
from importlib import reload
reload(galf)
reload(dis)

In [ ]:
# Define all strings here before running everything!!!

# Path to cgmsquared_full_galaxy_table.fits
path_to_full_galaxy_table = './cgmsquared_full_galaxy_table.fits'

# Name of the qso (format: JXXXX+XXXX)
qso_name = 'JXXXX+XXXX'

# The maximum angular distance from the qso to the galaxy
max_ang_dist = 0.0333333

# The maximum impact parameter considered
max_rho_impact = 500 # kpc

# The maximum redshift considered
max_z = 0.65

# Initial guess of the integrated magnitude for all galaxies
int_mag = 16

# Path to the UNFLATTENED fits file
unflat_data = 'JXXXX+XXXX_GEM.fits'

# For finding galaxy x and y coordinates, add an offset if needed
X_pixels = 0 # pixels
Y_pixels = 0 # pixels

# Name of the FLATTENED fits file you want to make
flat_data = 'JXXXX+XXXX_GEM_flat.fits'

# Name of the final fits file
final_fits_name = 'JXXXX+XXXX_GEM_output.fits'

# Name of the input files to run
input_file_name = 'JXXXX+XXXX_GEM_input.txt'

# Name of the input PSF (if any). Otherwise, leave as 'none'
psf = 'none'

# Name of the mask (if any). Otherwise, leave as 'none'
mask = 'none'

# Photometric zeropoint
ZP = 27.85

# Pixel scale
pix_scale = 'X.XX X.XX'

# Length of the square region to get small galaxy image
region_length = 100

# Best fit parameter CSV file
final_csv = 'JXXXX+XXXX_GEM_bestfit.csv'

In [ ]:
# to make the flattened fits file from the original:
with fits.open(unflat_data) as hdulist:
    hdu = hdulist[1]
    phdu = fits.PrimaryHDU(hdu.data,
                           header=hdu.header)
    phdu.writeto(flat_data,
                 overwrite=True)

In [ ]:
# Gets the cgmsquared_full_galaxy_table.fits file and turns it into a Pandas DataFrame
# See Function 1 in galfit_functions_v19_22.py

full_galaxy_df = galf.make_full_galaxy_table(path=path_to_full_galaxy_table)

In [ ]:
# Filters out unwanted galaxies from the table 
# See Function 2 in galfit_functions_v19_22.py

df_nearqso = galf.select_near_qso(
    full_galaxy_df=full_galaxy_df,
    qso_name=qso_name,
    max_ang_dist=max_ang_dist,
    max_rho_impact=max_rho_impact,
    max_z=max_z,
    include_SFE=True
)

In [ ]:
df_nearqso

In [ ]:
# Convert world coordinates for each galaxy to image pixel coordinates
# See Function 3 in galfit_functions_v19_22.py

coords = galf.get_pix_coords(
    path=unflat_data,
    df_nearqso=df_nearqso,
    x_offset= X_pixels,
    y_offset= Y_pixels
)

In [ ]:
coords

In [ ]:
# Prepare the models which will go into the input files to run
# See Function 4 in galfit_functions_v19_22.py

models = galf.make_models(coords=coords,
                          int_mag=int_mag)

In [ ]:
# Takes the average of the image's pixel values so we can assume that value for the background
# See Function 5 in galfit_functions_v19_22.py

sky = galf.make_sky_model(path=flat_data)

In [ ]:
# Makes all of the input files (one for each galaxy)
# See Function 6 in galfit_functions_v19_22.py

galf.CreateFile(
    Iimg=flat_data,
    coords=coords,
    models=models,
    psf=psf,
    mask=mask,
    final_fits=final_fits_name,
    input_file=input_file_name,
    ZP=ZP,
    scale=pix_scale,
    size=region_length / 2,
    sky=sky
)

CreateFile always returns a 0. If you look in your working directory you should see the input file which you can now put into galfit.

In [ ]:
# Creates a string which you can copy/paste into the terminal (in the same directory) to run all files
# See Function 7 in galfit_functions_v19_22.py

put_into_terminal = galf.run_all(coords=coords,
                                 input_file=input_file_name)

print(put_into_terminal)

# Copy and paste the string below into the terminal in the same directory as all your input files

# Run Alexandre's code for displaying galaxies to check the goodness of fits

In [ ]:
bad = []
double_method = []
internal_struct = []
edited = []
multi_gal = []

dis.display_galaxies('./', final_fits_name.replace('.fits', ''), input_file_name.replace('.txt', ''),
                    fieldname = qso_name,
                    mask_name = mask,
                    Bad_galaxy_list = bad,
                    galaxy_with_holes = double_method,
                    list_gal_with_Residual = internal_struct,
                    list_edited_gal = edited,
                    list_gal_Multi_galaxy = multi_gal,
                    Half_light_minimum = 3,
                    Info_option = True,
                    Flag_option = False)

# Make models look as good as humanly possible before proceeding

In [ ]:
# Read in the information from the fit.log file
# See Function 8 in galfit_functions_v19_22.py
# Remember to define coords above!

best_fit_table = galf.extract_best_fit_param(path='./fit.log',
                                             df_nearqso=df_nearqso,
                                             coords=coords)
best_fit_table

# Finding azimuthal angle $\phi$

In [ ]:
# See Function 10 in galfit_functions_v19_22.py

best_fit_table = galf.find_phi(unflattened_data=unflat_data,
                               near_qso_df=df_nearqso,
                               best_fit_table=best_fit_table)
best_fit_table

# Finding inclination angle $i$

In [ ]:
best_fit_table = galf.find_inclination(best_fit_table=best_fit_table)
best_fit_table

# Applying flags

If applicable, you need to enter galaxy numbers below before proceeding:

In [ ]:
# Enter galaxy field-specific numbers as list ([x, y, z, ...])
# Field-specific numbers start at 0 for all qso fields
# Field-specific numbers will be the first number after the qso name

bad_fits_in = [] # Bad fit galaxies
internal_struct_in = [] # Galaxies where there are weird patterns in disk (internal structure)
overlap_in = [] # Galaxies where other objects/galaxies overlap at least somewhat
pixel_thresh = 3 # Number of pixels to compare galaxy size to
edited_in = [] # Galaxies which had their fit parameters changed from their default values
double_model_in = [] # Galaxies which used double profiles on top of each other


In [ ]:
best_fit_table = galf.apply_flags(best_fit_table=best_fit_table,
                                  bad_fits_in=bad_fits_in,
                                  internal_struct_in=internal_struct_in,
                                  overlap_in=overlap_in,
                                  pixel_thresh=pixel_thresh,
                                  edited_in=edited_in,
                                  double_model_in=double_model_in)
best_fit_table

# Export Galfit's best fit parameters as a csv file

In [ ]:
best_fit_table.to_csv(final_csv, index=False)